In [18]:
import pandas as pd
import re
from dateutil.parser import parse
from datetime import datetime

In [17]:
df = pd.read_csv('Tweets.csv', sep=';')
# df.head()

/var/folders/zs/1zyd84vs1d7f727gjq7y2vpm0000gn/T/ipykernel_4216/343681958.py:1: DtypeWarning: Columns (1,2,4,10,14,15,18,19,20,21,22,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Tweets.csv', sep=';')


In [19]:
# Add Year-Month

def year(date_):
    try:
        return parse(date_).year
    except:
        return 0

def month(date_):
    try:
        return parse(date_).month
    except:
        return 0

df['year'] = df.created_at.apply(year)
df['month'] = df.created_at.apply(month)

In [22]:
df['year'].value_counts()

2022    103629
2021     37222
2020     20579
2019      7326
2018      4990
2017      3813
2016      2201
2015      1913
2014      1811
2013       964
2012       961
2011       407
2010        72
0            6
2009         5
Name: year, dtype: int64

In [24]:
# Select only tweets from 2022

df = df[df.year == 2022]

In [25]:
df.shape

(103629, 35)

In [26]:
# Remove @ usernames from tweets

def remove_usernames(text):
    return re.sub('@[\w]+','', str(text))

df.full_text = df.full_text.apply(remove_usernames)

In [27]:
# Remove links from tweets

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

df.full_text = df.full_text.apply(remove_URL)


In [5]:
# df.tail()

In [28]:
docs = df.full_text.to_list()
months = df.month.to_list()
# docs

In [31]:
# docs = docs[:10000]
len(docs), len(months)

(103629, 103629)

# Topic Modeling

In [33]:
from bertopic import BERTopic

In [34]:
# from sentence_transformers import SentenceTransformer
# from umap import UMAP
# from hdbscan import HDBSCAN

# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
# hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
#                         gen_min_span_tree=True,
#                         prediction_data=True)

In [37]:
# topic_model = BERTopic(
#     umap_model=umap_model,
#     hdbscan_model=hdbscan_model,
#     embedding_model=embedding_model,
#     vectorizer_model=vectorizer_model,
#     top_n_words=5,
#     language='english', #multilingual
#     calculate_probabilities=True,
#     verbose=True
# )

In [36]:
topic_model = BERTopic(language="multilingual",
                        calculate_probabilities=False,
                        verbose=True,
                        min_topic_size=75)

### Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 

In [38]:
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/3239 [00:00<?, ?it/s]

2022-12-20 00:01:49,375 - BERTopic - Transformed documents to Embeddings
2022-12-20 00:02:40,304 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2022-12-20 00:02:47,521 - BERTopic - Clustered reduced embeddings


### Extracting Topics

In [41]:
freq = topic_model.get_topic_info()
freq.head(25)

,Topic,Count,Name
0,-1,48197,-1_the_it_and_to
1,0,4988,0_compromise_kho__
2,1,4087,1_li_chi_rah_3la
3,2,3196,2_morocco_المغرب_moroccan_maroc
4,3,1967,3_txs_with__
5,4,1627,4_wili_بونو_يسطا_stefaniil
6,5,1461,5_الله_god_لله_شاء
7,6,1390,6_الله_اللهم_اللي_لي
8,7,1241,7_thank_thanks_merci_much
9,8,1196,8_two_try_when_do


In [43]:
topic_model.get_topic(22)  # Select the most frequent topic

[('devoxxma', 0.050264301064515286),
 ('meet', 0.03756976806240181),
 ('speakers', 0.026608794730338614),
 ('meeting', 0.02422760496062518),
 ('event', 0.02029447328134894),
 ('devfest', 0.01915221301180802),
 ('talk', 0.019002003589184694),
 ('was', 0.017197147883725606),
 ('forward', 0.01634031189337657),
 ('great', 0.016250539501232827)]

In [11]:
topic_model.topics_[:10]

[61, 81, -1, -1, 2, 34, -1, -1, -1, 2]

# Visualization
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

### Visualize Topics

After having trained our BERTopic model, we can iteratively go through perhaps a hundred topic to get a good understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. Instead, we can visualize the topics that were generated in a way very similar to LDAvis:

In [44]:
topic_model.visualize_topics(top_n_topics=75)

### Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [15]:
# topic_model.visualize_distribution(probs[200], min_probability=0.015)

### Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [45]:
topic_model.visualize_hierarchy(top_n_topics=75)

### Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [46]:
topic_model.visualize_barchart(top_n_topics=10)

In [78]:
topic_model.get_representative_docs(top)

{2: [' 2-1 for morocco we are gonna win nchalah',
  "  It is not any player's fault\nIt is powerfully OF Morocco",
  "   😂😂😂 c'est les Marocains qui ont libéré leur terre 😭😭",
  'Game over. Give us the world cup.',
  'One of my best moment in this world cup, I love this energy 🤣🤣🤣🤣 #FIFAWorldCup #QatarWorldCup2022 ',
  '#السعوديه_الارجنتين #FIFAWorldCup  #كأس_العالم_2022 #كأس_العالم_قطر_2022 ',
  'صباح الخير، فخور بأداء الاسود القتالي، ديما مغرب 🇲🇦، من باڤاريا. ',
  '#DimaMaghrib 💪🇲🇦 ',
  ' Inshallah #DimaMaghrib 🇲🇦🇲🇦🇲🇦',
  '😂😂😂😂 ',
  ' 😂😂😂😂',
  '😂😂😂😂',
  ' We are proud of you',
  '🇲🇦🇲🇦 Drari really made us proud ',
  "No matter what we love and still encourage our Morroco ♥️🇲🇦🍀 ... Thank you for everything ♥️😍 can't be more proud ♥️ ",
  '  لا لا معتقدش قد ولى ذاك الزمن.. اليوم صحوة للكرة المغربية يا إنا يواصل المسير مع الوداد يا إما الاحتراف في اوربا',
  'المسلم القويم لا يتشفى فالناس \nفرنسي كوري جزائري ... ',
  ' و حنا مالنا ؟؟ \nC quoi la relation entre la production du phoosphate

## Predict

We can predict what topic any arbitrary text belongs to using the fitted model. We can accomplish this by calling the transform() method. The code below demonstrates this and uses a made up headline for a news article

In [86]:
# text = "ياله خديت التيكي ديالي باش نحضر بلابلاكونف اللي منضمة من طرف الكومينوتي غيغس بلابلا. اش مازال كتسنى، بالي قبل ما يسالي و قطع ورقتك حتى نتا"
text = 'One of the sessions that I LOVED today ! Design thinking enables us to create Solutions that are Desirable, Feasible and Viable !'

preds, probs = topic_model.transform(text)
print(preds)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-12-20 00:45:31,887 - BERTopic - Reduced dimensionality
2022-12-20 00:45:31,906 - BERTopic - Predicted clusters


[-1]


In [87]:
topic_model.get_topic(-1)

[('in the', 0.0034637327245780353),
 ('of the', 0.0034136140857753423),
 ('is the', 0.0024549142647816275),
 ('you can', 0.0023382883893685474),
 ('this is', 0.002324523104440822),
 ('if you', 0.0021950792753658572),
 ('for the', 0.0021695064518143783),
 ('on the', 0.0021192969416989286),
 ('to the', 0.0021112693398216936),
 ('to be', 0.0020038755018632025)]

## Similarity Check

In [93]:
topic_model.visualize_heatmap(topics=[9], top_n_topics=5)

In [100]:
similar_topics, similarity = topic_model.find_topics("java", top_n=5)

for similar_topic in similar_topics:
    print(similar_topic)
    print(topic_model.get_topic(similar_topic))

1
[('chi haja', 0.002403209856644628), ('check in', 0.0023392539902489296), ('dakchi li', 0.0022501946035374735), ('wld l97ba', 0.0022394773061670077), ('chi wahd', 0.0016351191917936597), ('gt gt', 0.0015859771492811405), ('nass li', 0.0013944784028937472), ('rah machi', 0.0013161057360451056), ('chi nhar', 0.001172156522310266), ('wlad l9hab', 0.001172156522310266)]
6
[('wili wili', 0.00717318643804802), ('wili wili wili', 0.005021936369559749), ('سير سير', 0.004850255816954791), ('بونو بونو', 0.00396193805966494), ('stefaniil stefaniil', 0.00396193805966494), ('تا تا', 0.00396193805966494), ('this is', 0.003021619885797343), ('اسود شكرا', 0.0027447374973312453), ('هاد التويتر', 0.0027447374973312453), ('بونو بونو بونو', 0.0027447374973312453)]
3
[('شاء الله', 0.010167177483008931), ('ان شاء الله', 0.00644266747196232), ('ان شاء', 0.006429138503943388), ('إليه راجعون', 0.005492289250313264), ('لا حول', 0.005210100407116515), ('الحمد لله', 0.004577221365173647), ('إنا لله', 0.00453318

In [94]:
similar_topics, similarity = topic_model.find_topics("maroc", top_n=5)
topic_model.get_topic(similar_topics[0])

[('in morocco', 0.009216398962223244),
 ('le maroc', 0.006571214646511338),
 ('du maroc', 0.0049267704939113975),
 ('world cup', 0.0048608966628348495),
 ('proud of', 0.004563850055981638),
 ('في المغرب', 0.004410392314072939),
 ('au maroc', 0.004336240328345411),
 ('the moroccan', 0.003886667962326814),
 ('de la', 0.0036400942566339424),
 ('morocco is', 0.003488059041612869)]

# Topics over Time

Although extracting the topics and their representation is interesting, we are missing some dimensional information. For example, some topics might not be relevant anymore or some are gaining traction over the last years. That can be vital information when making decisions. 

Here, we will model the topics over the years. For each topic and timestep, we calculate the c-TF-IDF representation. This will result in a specific topic representation at each timestep without the need to create clusters from embeddings as they were already created. However, topics can be regarded as evolutionary entities that evolve and change over time. This means that a topic representation at timestep *t* should be related to its representation at timesetps *t-1* and *t+1*. To model this evolutionary trend, we average its c-TF-IDF representation with that of the c-TF-IDF representation at timestep *t-1*. This is done for each topic representation allowing for the representations to evolve over time.

In [49]:
topics_over_time = topic_model.topics_over_time(docs, months)

12it [00:17,  1.49s/it]


In [53]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=75, width=1000, height=700)

In [55]:
topic_model.visualize_topics_over_time(topics_over_time, width=1000, height=700,
                                        topics=[9, 25, 53, 63, 66, 74])

### Visualize Topic Similarity

Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [57]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

### Visualize Term Score Decline

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.

In [58]:
topic_model.visualize_term_rank()

# Topic Representation

After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

### Update Topics

When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 

In [61]:
topic_model_updated = topic_model
topic_model_updated.update_topics(docs, n_gram_range=(2, 3))

In [62]:
topic_model_updated.get_topic(0)   # We select topic that we viewed before

[('kho compromise', 9.336444248771908),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

### Topic Reduction

We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, is that you can decide the number of topics after knowing how many are actually created. It is difficult to predict before training your model how many topics that are in your documents and how many will be extracted. Instead, we can decide afterwards how many topics seems realistic:

In [63]:
topic_model_updated.reduce_topics(docs, nr_topics=20)

2022-12-20 00:29:53,633 - BERTopic - Reduced number of topics from 174 to 21


In [25]:
# Access the newly updated topics with:
# print(topic_model.topics_)

# Search Topics

After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [76]:
similar_topics, similarity = topic_model_updated.find_topics("tech", top_n=5); similar_topics

[-1, 17, 1, 15, 14]

In [77]:
topic_model.get_topic(15)

[('de la', 0.0066625337797131725),
 ('el pueblo', 0.004386767240335964),
 ('the president', 0.0033105123762771083),
 ('in the', 0.0026250538681166793),
 ('war is', 0.00259649347288125),
 ('to the', 0.002571936999714065),
 ('of the', 0.002441336631996827),
 ('dans la', 0.002231229065621415),
 ('la politique', 0.002225565833898214),
 ('they are', 0.00220678244623804)]

# Model serialization
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model_all")	

In [19]:
# Load model
my_model = BERTopic.load("my_model_all")

# Embedding Models

The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

### Sentence-Transformers

You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:


In [39]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)